In [2]:
from bertviz import head_view
from IPython.display import clear_output
from transformers import BertTokenizer, BertModel
import train_bert
import torch
import torch.nn as nn
import math

I0312 15:37:39.403473 140168851584768 file_utils.py:38] PyTorch version 1.2.0 available.
Using TensorFlow backend.


In [3]:
# 在 jupyter notebook 裡頭顯示 visualzation 的 helper
def call_html():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

clear_output()

In [4]:
class gelu(nn.Module):
    
    def __init__(self):
        super(gelu, self).__init__()

    def forward(self, x):
        cdf = 0.5 * (1.0 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        return x * cdf

    
class MutiLabelModel(nn.Module):
    
    def __init__(self, encoder, emb_size=1024, out_size=5, ce_size=23, hidden=256): # hidden=256
        super(MutiLabelModel, self).__init__()
        
        self.encoder = encoder
        self.fn_size = emb_size
        
        self.out_fn = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.fn_size, self.fn_size//2),
            gelu(),
            nn.Dropout(0.2),
            nn.Linear(self.fn_size//2, out_size),
        )
        
    def forward(self, inp, seg, cat_emb=None, cls_loc=0): # , inp_title, seg_inp_title, cls_loc=0):

        embs = self.encoder(inp, seg)[0] # [batch, seq, hidden]
        outputs = embs[:, cls_loc, :]
        outputs = self.out_fn(outputs)

        return outputs

In [5]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        attention = model(input_ids, token_type_ids)[-1]
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        attention = model(input_ids)[-1]
        sentence_b_start = None
    
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    print(attention)
    head_view(attention, tokens, sentence_b_start)

In [6]:
model_version = 'bert-base-chinese'

# load pre-trained bert
print("\nloading bert...")
tokenizer = BertTokenizer.from_pretrained(model_version)
model = BertModel.from_pretrained(model_version, output_attentions=True)   


loading bert...


I0312 15:37:44.568570 140168851584768 tokenization_utils.py:418] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /home/dl-ismp-mh/han/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0312 15:37:45.490202 140168851584768 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /home/dl-ismp-mh/han/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0312 15:37:45.492957 140168851584768 configuration_utils.py:290] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning

In [7]:
ck_path = '/home/dl-ismp-mh/han/Trauma_Ontology/test_ck.pt'

state = torch.load(ck_path)
model.load_state_dict(state['encoder'])

<All keys matched successfully>

In [8]:
# 情境 1 的句子
sentence_a = "病患來診為昨晚工作時被纜繩割傷，現左手第四指創傷性截肢入。"
sentence_b = ""

# 得到 tokens 後丟入 BERT 取得 attention
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()

# 交給 BertViz 視覺化
head_view(attention, tokens)

# 注意：執行這段程式碼以後只會顯示下圖左側的結果。
# 為了方便你比較，我把情境 2 的結果也同時附上

## layer 2[CLS] 注意其他單詞可預測單詞
## layer 2 注意下一個字
## layer 2 注意相同/相關的詞
## layer 6 注意上一個字
## Layer10 注意其他句子中的相同/相關詞


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>